In [2]:
import matplotlib as mt
import pandas as pd
import pylab as pl
import numpy as np
import scipy as sc
import scipy.optimize as opt

data = pd.read_csv("Test_data.csv", header=2,low_memory=False)
data = data.convert_objects(convert_numeric=True)
data_c = pd.DataFrame()

## Prepare constant Values
RFPF = 0.0133
CH4_RF = 1.1
Tchiller = 7
Pamb = 98.0918852
Pchiller = Pamb + 23.75
xCO2intdry = 0.000375
xCO2dildry = 0.000375
xTHC_THC_FID_init = 1
print("Initialization finished!")

Initialization finished!


In [3]:
##### First Level of Calculation #####
data_c["E_CO2D2"] = data.E_CO2D2*10000 # % --> ppm
data_c["E_COHD2"] = data.E_COHD2*10000 # % --> ppm
data_c["E_NOXD2"] = data.E_NOXD2
data_c["E_THCW2"] = data.E_THCW2
data_c["xCH4wet"] = data.E_CH4W2/1000000 # ppm --> mol/mol
data_c["C_FRAIRWS"] = data.C_FRAIRWS
data_c["BARO Press"] = data.P_INLET*100 # in kPa
data_c["T_INLET"] = data.T_INLET
data_c["M_RELHUM"] = data.M_RELHUM
data_c["mfuel"] = data.M_FRFUEL*1000/3600 # kg/h --> g/s
print("First Level of Calculation finished!")

First Level of Calculation finished!


In [4]:
##### Second Level of Calculation #####
data_c["xCO2meas"] = data_c.E_CO2D2/1000000 # ppm --> mol/mol
data_c["xCOmeas"] = data_c.E_COHD2/1000000 # ppm --> mol/mol
data_c["xNOxmeas"] = data_c.E_NOXD2/1000000 # ppm --> mol/mol
data_c["xTHCmeas"] = data_c.E_THCW2/1000000 # ppm --> mol/mol
data_c["Molar Flow Wet"] = data_c.C_FRAIRWS*1000/3600 # kg/h --> g/s
data_c["maf_kgh (wet)"] = data_c.C_FRAIRWS
data_c["pH2O @ inlet"] = 10**(10.79574*(1-(273.16/(273.15+data_c.T_INLET)))-5.028*np.log10((273.15+data_c.T_INLET)/273.16)+0.000150475*(1-10**(-8.2969*(((273.15+data_c.T_INLET)/273.16)-1)))+0.00042873*(10**(4.76955*(1-(273.16/(273.15+data_c.T_INLET))))-1)-0.2138602)  ## Reference: CFR 1065.564
data_c["xH2O"] = data_c.M_RELHUM*data_c.get("pH2O @ inlet")/(data_c.get("BARO Press")*100) ## Reference: CFR 1065.564
data_c["xTHC[THC_FID]cor"] = data_c.E_THCW2-xTHC_THC_FID_init ## Reference: CFR 1065.660
print("Second Level of Calculation finished!")

Second Level of Calculation finished!


In [5]:
##### Third Level of Calculation #####
data_c.E_THCW2 = data_c.get("xTHC[THC_FID]cor") ## Reference: CFR 1065.660
data_c["xNO2meas"] = data_c.xNOxmeas*0
data_c["xNOmeas"] = data_c.xNOxmeas*1
data_c["xTHCwet"] = data_c.xTHCmeas
data_c["Intake Air flow"] = data_c.get("Molar Flow Wet")/28.96 # g/sec --> mol/sec
data_c["Mmix"] = 28.96559*(1-data_c.xH2O)+18.01528*data_c.xH2O
data_c["xH2Oint"] = data_c.xH2O
data_c["xH2Odil"] = data_c.xH2O

# Constant Channels
c = np.zeros((len(data_c),1))
c.fill(3.5)
data_c["xH2Ogas"] = c
c.fill(13.875389)
data_c["Molar Mass of HC Emissions"] = c
c.fill(28.0101)
data_c["Molar Mass of CO Emissions"] = c
c.fill(46.0055)
data_c["Molar Mass of NOx Emissions (as NO2)"] = c
c.fill(44.0095)
data_c["Molar Mass of CO2 Emissions"] = c
c.fill(13.875389)
data_c["Molar Mass of NMHC Emissions"] = c
c.fill(0.866)
data_c["Wc"] = c
print("Third Level of Calculation finished!")

Third Level of Calculation finished!


In [6]:
##### Fourth Level of Calculation #####
data_c["xNMHCwet"] = (data_c.xTHCwet-data_c.xCH4wet*CH4_RF)/(1-RFPF*CH4_RF) ## Reference: CFR 1065.660
data_c["nint (Intake Air Flow)"] = data_c.get("Molar Flow Wet")/data_c.Mmix
data_c["xH2Ointdry"] = data_c.xH2Oint/(1-data_c.xH2Oint)
data_c["xH2Odildry"] = data_c.xH2Odil/(1-data_c.xH2Odil)
print("Fourth Level of Calculation finished!")

Fourth Level of Calculation finished!


In [7]:
##### Fifth Level of Calculation #####
data_c["xCO2int"] = xCO2intdry/(1+data_c.xH2Ointdry)
data_c["xO2int"] = ((0.20982-xCO2intdry)/(1+data_c.xH2Ointdry)) 
data_c["xCO2dil"] = xCO2dildry/(1+data_c.xH2Odildry)
print("Fifth Level of Calculation finished!")

Fifth Level of Calculation finished!


In [8]:
##### Circle Calculation #####
def f(variables):
    # Channels in Formulas
    #A = data.xdil/exh -- G
    #B = data.xH2Oexh -- H
    #C = data.xCcombdry -- I
    #D = data.xH2Oexhdry -- J
    #E = data.xdil/exhdry -- K
    #F = data.xint/exhdry -- L
    #G = data.xraw/exhdry -- M
    #H = data.xH2OCOmeas -- AC
    #I = data.xH2OTHCmeas -- AD
    #J = data.xH2ONOxmeas -- AE
    #K = data.xH2ONO2meas -- AF
    #L = data.xH2OCO2meas -- AH
    #M = data.xCOdry -- AI
    #N = data.xTHCdry -- AJ
    #O = data.xNOxdry -- AK
    #P = data.xNO2dry -- AL
    #Q = data.xCO2dry -- AN
    #R = data.xH2dry -- AO    
    
    # Constant Values    
    alpha = 1.84
    beta = 0
    gamma = 0
    delta = 0
    h2ogas = 3.5    
    
    # Unknown Variables
    (A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R) = variables
        
    # Known Variables
    S = data_c.xCO2dil[i]
    T = data_c.xCO2int[i]
    U = alpha
    V = data_c.xH2Odil[i]
    W = data_c.xH2Oint[i]
    X = data_c.xO2int[i]
    Y = beta
    Z = gamma
    AA = delta
    AB = data_c.xCOmeas[i]
    AC = data_c.xTHCmeas[i]
    AD = data_c.xNOxmeas[i]
    AE = data_c.xNO2meas[i]
    AF = h2ogas
    AG = data_c.xCO2meas[i]
    
    # Equations to solve
    eq1 = 1-(G/(1+D))-A
    eq2 = D/(1+D)-B
    eq3 = Q+(M)+(N)-(S*E)-(T*F)-C
    eq4 = ((U/2)*(C-N))+V*E+W*F-R-D 
    eq5 = A/(1-B)-E
    eq6 = (1/(2*X))*(((U/2)-Y+2+(2*Z))*(C-N)-(M-O-(2*P)+R))-F
    eq7 = 0.5*(((U/2)+Y+AA)*(C-N)+((2*N)+M-P+R))+F-G
    eq8 = AB/(1-H)-M ## Reference: CFR 1065.659
    eq10 = AC/(1-I)-N ## Reference: CFR 1065.659
    eq11 = B-I
    eq12 = AD/(1-J)-O ## Reference: CFR 1065.659
    eq14 = AE/(1-K)-P ## Reference: CFR 1065.659
    eq16 = (M*(D-V*E))/(AF*(Q-S*E))-R ## Reference: CFR 1065.659
    eq17 = AG/(1-L)-Q ## Reference: CFR 1065.659
    
    if mode==0:
        eq9 = bs-H    
        eq13 = bs-J
        eq15 = bs-K
        eq18 = bs-L
    else:
        eq9 = B-H    
        eq13 = B-J
        eq15 = B-K
        eq18 = B-L
    
    return [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,eq10,eq11,eq12,eq13,eq14,eq15,eq16,eq17,eq18]

i = 0
mode = 0
b = np.zeros((len(data_c),18))
bs = 0.00821722713286476 # Has to be calculated

while i < len(data_c)-1:
    i +=1  
    g = 0.5 # First guess for iteration start

    solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
    
    if solution[2]<bs:
        mode = 1
        solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
        mode = 0
    b[:][i] = solution

df = pd.DataFrame(b,columns =['xdil/exh','xH2Oexh','xCcombdry','xH2Oexhdry','xdil/exhdry','xint/exhdry','xraw/exhdry','xH2OCOmeas','xH2OTHCmeas','xH2ONOxmeas','xH2ONO2meas','xH2OCO2meas','xCOdry','xTHCdry','xNOxdry','xNO2dry','xCO2dry','xH2dry'])
data_c = pd.concat([data_c,df],axis=1)
print("Circle Calculation finished!")
## 05/06/2015 - Anton lang, Lang@FEV.com

Circle Calculation finished!


In [9]:
##### Calculation of remaining values #####
data_c["xH2ONOmeas"] = data_c.xH2ONO2meas
data_c["xNOdry"] = data_c.xNOmeas/(1-data_c.xH2ONOmeas) ## Reference: CFR 1065.659
data_c["xCO2dry"] = data_c.xCO2meas/(1-data_c.xH2OCO2meas) ## Reference: CFR 1065.659
data_c["xH2dry"] = (data_c.xCOdry*(data_c.xH2Oexhdry-data_c.xH2Odil*data_c.get("xdil/exhdry")))/(data_c.xH2Ogas*(data_c.xCO2dry-data_c.xCO2dil*data_c.get("xdil/exhdry")))
data_c["xCOwet"] = data_c.xCOmeas*((1-data_c.xH2Oexh)/(1-data_c.xH2OCOmeas))
data_c['xNOxwet'] = data_c.xNOxmeas*((1-data_c.xH2Oexh)/(1-data_c.xH2ONOxmeas))
data_c["xNO2wet"] = data_c.xNO2meas*((1-data_c.xH2Oexh)/(1-data_c.xH2ONO2meas))
data_c["xNOwet"] = data_c.xNOmeas*((1-data_c.xH2Oexh)/(1-data_c.xH2ONOmeas))
data_c["xCO2wet"] = data_c.xCO2meas*((1-data_c.xH2Oexh)/(1-data_c.xH2OCO2meas))
data_c["xNOxcorrwet"] = data_c.xNOxwet*(18.84 *data_c.xH2O + 0.68094) ## Reference: CFR 1065.670

data_c["nexh"] = data_c.get("nint (Intake Air Flow)")/(1+((data_c.get("xint/exhdry")-data_c.get("xraw/exhdry"))/(1+data_c.xH2Oexhdry)))
data_c["Mass_THC"] = data_c.xTHCwet*data_c.nexh*data_c.get("Molar Mass of HC Emissions")
data_c["Mass_CO"] = data_c.xCOwet*data_c.nexh*data_c.get("Molar Mass of CO Emissions")
data_c["Mass_NOx"] = data_c.xNOxcorrwet*data_c.nexh*data_c.get("Molar Mass of NOx Emissions (as NO2)")
data_c["Mass_CO2"] = data_c.xCO2wet*data_c.nexh*data_c.get("Molar Mass of CO2 Emissions")
data_c["Mass_NMHC"] = data_c.xNMHCwet*data_c.nexh*data_c.get("Molar Mass of NMHC Emissions")

data_c.to_csv("Calculation_Report.csv")
print("Calculation finished and Report saved!")

Calculation finished and Report saved!


In [10]:
# Emissions in total
sum_THC = data_c.Mass_THC.sum(skipna = True)
sum_CO = data_c.Mass_CO.sum(skipna = True)
sum_NOx = data_c.Mass_NOx.sum(skipna=True)
sum_CO2 = data_c.Mass_CO2.sum(skipna=True)
sum_NMHC = data_c.Mass_NMHC.sum(skipna=True)

In [11]:
sum_THC

6.4206272155960997